In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


Import statistical packages

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb


In [5]:
Nbacombined = pd.read_csv("Nba_Combined.csv")
Nbacombined = Nbacombined.dropna()

In [6]:
train  = Nbacombined[Nbacombined['Year']<2024]
test = Nbacombined[Nbacombined['Year']==2024]

Nbacombined.columns

Index(['Player', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA',
       '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'USG%', 'TS%',
       'PTS_per_TSA', 'FT_rate', '3P_rate', 'AST_TO', 'TOV%', 'TRB%_approx',
       'STL_per_min', 'BLK_per_min', 'STL%', 'BLK%', 'Raw_EFF', 'PF_per_min',
       'Load', 'Creation_proxy', 'PTS_log', 'AST_log', 'TRB_log', 'STL_log',
       'BLK_log', 'TOV_log'],
      dtype='object')

In [15]:
features = [
    'Age', 'G', 'MP', 'FG%',
    'eFG%', 'FT%',   'Year', 'USG%',
    'FT_rate', '3P_rate', 'Raw_EFF',
    'Load', 'Creation_proxy'
]
target = ['PTS']

In [16]:
X_train = train[features]
y_train = train[target]

X_test = test[features]
y_test = test[target]



In [17]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
reg = xgb.XGBRegressor(n_estimators=1000, learning_rate=.01) 
reg.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    verbose=100
)


[0]	validation_0-rmse:6.08270
[100]	validation_0-rmse:2.47601
[200]	validation_0-rmse:1.23702
[300]	validation_0-rmse:0.85816
[400]	validation_0-rmse:0.74645
[500]	validation_0-rmse:0.70448
[600]	validation_0-rmse:0.68106
[700]	validation_0-rmse:0.66840
[800]	validation_0-rmse:0.66048
[900]	validation_0-rmse:0.65469
[999]	validation_0-rmse:0.65085


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.01, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [18]:
y_pred = reg.predict(X_test)

In [19]:
test_with_predictions = test[["Player", "PTS"]].copy()
test_with_predictions['PTS_pred'] = y_pred

In [20]:
test_with_predictions


,Player,PTS,PTS_pred
0,A.J. Green,4.5,4.305826
2,A.J. Lawson,3.2,3.133325
5,AJ Griffin,2.4,2.425513
13,Aaron Gordon,13.9,12.470241
21,Aaron Holiday,6.6,6.279456
...,...,...,...
3821,Zach LaVine,19.5,19.446463
3831,Zavier Simpson,6.0,6.819988
3835,Zeke Nnaji,3.2,3.168575
3845,Ziaire Williams,8.2,8.133763


In [21]:
nba_2025 = pd.read_csv('new_nba2025.csv')
X_2025 = nba_2025[features]
y_2025_pred = reg.predict(X_2025)
nba_2025_pred = nba_2025[["Player","PTS"]].copy()
nba_2025_pred["PTS_pred"] = y_2025_pred
nba_2025_pred.to_csv("new_nba_2025_pred.csv", index = "Player")